Module importieren

In [244]:
import matplotlib.pyplot as plt
import pandas as pd
#import pd as pd
import seaborn as sns
from datetime import datetime
from fitter import Fitter
import numpy as np
from scipy import stats
import statsmodels.api as sm
from tabulate import tabulate

Daten laden und säubern

In [245]:
def open_csv_to_df(directory):
    df = pd.read_csv(directory)
    df = df.iloc[0:,:4]
    df.columns = ["Date", "Time", "Mass_(kg)", "Velocity_(m/s)"]
    df.insert(4, "Energy_(kJ)", "")
    df.insert(5, "Date_and_Time", "")
    df.insert(6, "Time_delta_(h)", "")
    for i in range(len(df)):
        df.iloc[i,4] = 0.5 * float(df.iloc[i,2]) * float(df.iloc[i,3]) ** 2 / 1000 #calculation of Energy_(kJ)
    df = df.sort_values(['Date', 'Time']).reset_index(drop=True)
    df = df.dropna()

    for i in range(len(df)):
        date_raw = df.iloc[i,:].Date +" " +df.iloc[i,:].Time
        date1 = datetime.strptime(date_raw, '%Y-%m-%d %H:%M')
        df.iloc[i,5] = date1

    for i in range(len(df)-1):
        date1 = df.iloc[i,5]
        date2 = df.iloc[i+1,5]
        time_delta = date2 - date1
        time_delta = (time_delta.days*24) + (time_delta.seconds//3600)
        df.iloc[i+1,6] = time_delta

    #Fitter kann keine NAs handhaben aus diesem Grund wird erste Beobachtung mit dem Median der Zeitabstände ersetzt.
    df.iloc[0,6] = 0
    df['Time_delta_(h)'] = df['Time_delta_(h)'].astype('int')
    df.iloc[0,6] = df["Time_delta_(h)"].median()

    return df #this is the table with all stones combined

In [246]:
df1 = open_csv_to_df("./out_1.csv")
df2 = open_csv_to_df("./out_2.csv")

Funktion für Fitting von Verteilungen

In [247]:
def fit_distribution(data):
    count = 0
    fitted_mass = Fitter(data)
    fitted_mass.fit()
    ks_summary = fitted_mass.summary()
    distributions = ks_summary.index.values
    print(ks_summary.iloc[:,-2:])
    fig, ax = plt.subplots(1,5,figsize=(25, 5))
    table = [['Distribution', 'Parameters']]

    for i in distributions:
        param = fitted_mass.fitted_param[i]
        table.append([i, param])
        dist_eval = eval('stats.' + i + '.rvs(*param, size = 1000)')
        sm.qqplot_2samples(data, dist_eval, xlabel = i.capitalize() + ' Distribution', ylabel = 'Sample Distribution', line = '45' ,ax = ax[count])
        count += 1
    plt.show()
    print(tabulate(table, headers = 'firstrow'))

Zone 1

In [248]:
#fit_distribution(df1['Mass_(kg)'])

In [249]:
#fit_distribution(df1["Velocity_(m/s)"])

In [250]:
#fit_distribution(df1["Time_delta_(h)"])

Zone 2

In [251]:
#fit_distribution(df2['Mass_(kg)'])

In [252]:
#fit_distribution(df2["Velocity_(m/s)"])

In [253]:
#fit_distribution(df2["Time_delta_(h)"])

Monte Carlo

In [254]:
sim_count = 2_000_000


Zone 1

In [255]:
# Masse
param = stats.gamma.fit(df1['Mass_(kg)'])
zone1_mass = stats.gamma.rvs(*param, size = sim_count)

# Geschwindigkeit
param = stats.norm.fit(df1["Velocity_(m/s)"])
zone1_velocity = stats.norm.rvs(*param, size = sim_count)

# Zeitabstand
param = stats.expon.fit(df1["Time_delta_(h)"])
zone1_time_delta = stats.expon.rvs(*param, size = sim_count)

Zone 2

In [256]:
# Masse
param = stats.gamma.fit(df2['Mass_(kg)'])
zone2_mass = stats.gamma.rvs(*param, size = sim_count)

# Geschwindigkeit
param = stats.norm.fit(df2["Velocity_(m/s)"])
zone2_velocity = stats.norm.rvs(*param, size = sim_count)

# Zeitabstand
param = stats.expon.fit(df2["Time_delta_(h)"])
zone2_time_delta = stats.expon.rvs(*param, size = sim_count)

In [257]:
df1_sim = pd.DataFrame()
df1_sim.insert(0, "Mass_(kg)", zone1_mass)
df1_sim.insert(1, "Velocity_(m/s)", zone1_velocity)
df1_sim.insert(2, "Energy_(kJ)", "")
df1_sim.insert(3, "Time_delta_(h)", zone1_time_delta)


df1_sim["Time_delta_(h)"] = zone1_time_delta.round(4)
df1_sim["Time_delta_(h)"][0] = 0

df1_sim = df1_sim.dropna()
df1_sim["Energy_(kJ)"] = (0.5 * df1_sim["Mass_(kg)"] * (df1_sim["Velocity_(m/s)"] ** 2)) / 1000
df1_sim

C:\Users\logan\AppData\Local\Temp\ipykernel_14264\3105050015.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sim["Time_delta_(h)"][0] = 0


,Mass_(kg),Velocity_(m/s),Energy_(kJ),Time_delta_(h)
0,776.078557,6.542090,16.607671,0.0000
1,74.926867,7.796504,2.277233,2.0330
2,91.657202,7.910719,2.867930,27.2290
3,76.817078,8.159269,2.556997,8.2299
4,1955.008136,7.306632,52.185880,57.6314
...,...,...,...,...
1999995,293.860408,10.673671,16.739356,28.3564
1999996,311.203353,6.550015,6.675731,49.8829
1999997,1474.018473,9.621554,68.228111,54.0947
1999998,31.224902,7.819839,0.954700,86.1539


In [258]:
max(df1_sim["Energy_(kJ)"])

796.1998375302187

In [259]:
87 % 24

15

In [260]:
87 - 24 -24 -24

15

In [261]:
x = pd.DataFrame()
x.insert(0, "Mass_(kg)","")
x.insert(1, "Velocity_(m/s)","")
x.insert(2, "Energy_(kJ)","")
x.insert(3, "Time_delta_(h)","")

x = x.append(df1_sim.iloc[1,:])
x = x.append(df1_sim.iloc[2,:])
print(x)

   Mass_(kg)  Velocity_(m/s)  Energy_(kJ)  Time_delta_(h)
1  74.926867        7.796504     2.277233           2.033
2  91.657202        7.910719     2.867930          27.229


C:\Users\logan\AppData\Local\Temp\ipykernel_14264\3055680985.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x = x.append(df1_sim.iloc[1,:])
C:\Users\logan\AppData\Local\Temp\ipykernel_14264\3055680985.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x = x.append(df1_sim.iloc[2,:])


In [262]:
def netztest(status_netz,gewicht_im_netz,aufprallenergie):

    if status_netz == True:
        return True

    elif gewicht_im_netz > 2000:
        if aufprallenergie > 500:
            return True
        else:
            return False

    elif gewicht_im_netz <= 2000:
        if aufprallenergie > 1000:
            return True
        else:
            return False
    else:
        return False

In [263]:
netztest(True,2000,600)

True

In [264]:
max_delta = 24
current_delta = 0
netz = 0
steine_im_netz = 0
incidents = 0
incidents_stones = 0
stunden_total = 0
incident_status = False

for i in range(int(len(df1_sim))):
    iterated_delta = df1_sim["Time_delta_(h)"][i]
    iterated_energy = df1_sim["Energy_(kJ)"][i]

    stunden_total += iterated_delta


    if iterated_delta > max_delta:

        netz = df1_sim["Mass_(kg)"][i]
        steine_im_netz = 1
        incident_status = netztest(incident_status,0,iterated_energy)
        iterated_delta = iterated_delta % max_delta
        if iterated_delta > (max_delta - current_delta):

            current_delta = abs((max_delta - current_delta)-iterated_delta)

        elif iterated_delta <= (max_delta - current_delta):

            current_delta = current_delta + iterated_delta

    elif iterated_delta <= max_delta:

        if iterated_delta > (max_delta - current_delta):
            netz = df1_sim["Mass_(kg)"][i]
            steine_im_netz = 1
            incident_status = netztest(incident_status,0,iterated_energy)

            current_delta = abs((max_delta - current_delta)-iterated_delta)

        elif iterated_delta <= (max_delta - current_delta):
            incident_status = netztest(incident_status,netz,iterated_energy)
            netz = netz + df1_sim["Mass_(kg)"][i]
            steine_im_netz += 1
            current_delta = current_delta + iterated_delta

    if incident_status == True:
        incidents += 1
        incidents_stones += steine_im_netz

anzahl_steine = sim_count
prozent_netzbrechende_steine = (incidents_stones / len(df1_sim)) * 100
anzahl_sim_jahre = (stunden_total/8760)
anzahl_netzbrechende_steine_pro_jahr = (incidents_stones / anzahl_sim_jahre)



print(incidents_stones)

"""
print("Anzahl Steine:  ", anzahl_steine)
print("Totale_Stunden:  ", stunden_total)
print("Anzahl Jahre:", anzahl_sim_jahre)
print("Anzahl netzbrechende Steine:", incidents_stones)
print("Prozente netzbrechende Steine:  ", prozent_netzbrechende_steine,"%")
print("Anzahl Incidents pro Jahr:", anzahl_netzbrechende_steine_pro_jahr)
print("Prozente Incidents pro Jahr:", (anzahl_netzbrechende_steine_pro_jahr * prozent_netzbrechende_steine),"%")

#prüfen was passiert wenn netz gefallen, ob dann steine die fallen durchfallen oder nicht"""

2544723


'\nprint("Anzahl Steine:  ", anzahl_steine)\nprint("Totale_Stunden:  ", stunden_total)\nprint("Anzahl Jahre:", anzahl_sim_jahre)\nprint("Anzahl netzbrechende Steine:", incidents_stones)\nprint("Prozente netzbrechende Steine:  ", prozent_netzbrechende_steine,"%")\nprint("Anzahl Incidents pro Jahr:", anzahl_netzbrechende_steine_pro_jahr)\nprint("Prozente Incidents pro Jahr:", (anzahl_netzbrechende_steine_pro_jahr * prozent_netzbrechende_steine),"%")\n\n#prüfen was passiert wenn netz gefallen, ob dann steine die fallen durchfallen oder nicht'

In [265]:
df1_sim.iloc[0:9,:]

,Mass_(kg),Velocity_(m/s),Energy_(kJ),Time_delta_(h)
0,776.078557,6.542090,16.607671,0.0000
1,74.926867,7.796504,2.277233,2.0330
2,91.657202,7.910719,2.867930,27.2290
3,76.817078,8.159269,2.556997,8.2299
4,1955.008136,7.306632,52.185880,57.6314
5,82.609276,7.837113,2.536945,6.5988
6,467.858907,9.141404,19.548379,41.8856
7,344.323317,7.718375,10.256241,7.0556
8,109.694058,9.201928,4.644198,29.3851


In [266]:
"""
df_test = df1_sim.iloc[0:9,:]
df_test["Mass_(kg)"][3]  = 1500
df_test["Energy_(kJ)"][4]  = 3000
df_test["Time_delta_(h)"][5]  = 1
df1_sim = df_test
df1_sim"""

'\ndf_test = df1_sim.iloc[0:9,:]\ndf_test["Mass_(kg)"][3]  = 1500\ndf_test["Energy_(kJ)"][4]  = 3000\ndf_test["Time_delta_(h)"][5]  = 1\ndf1_sim = df_test\ndf1_sim'

In [270]:
max_delta = 24
current_delta = 0
stunden_total = 0
masse_im_netz = 0
status_netz = False
incident_steine = 0
steine_im_netz = 0

for i in range(int(len(df1_sim))):
    iterated_delta = df1_sim["Time_delta_(h)"][i]
    iterated_energy = df1_sim["Energy_(kJ)"][i]

    stunden_total += iterated_delta


    if iterated_delta > max_delta:
        if status_netz:
            incident_steine += steine_im_netz
        steine_im_netz = 1
        status_netz = netztest(False,0,iterated_energy)
        masse_im_netz = df1_sim["Mass_(kg)"][i]

        iterated_delta = iterated_delta % max_delta


        if iterated_delta > (max_delta - current_delta):

            current_delta = abs((max_delta - current_delta)-iterated_delta)


        elif iterated_delta <= (max_delta - current_delta):

            current_delta = current_delta + iterated_delta

    elif iterated_delta <= max_delta:

        if iterated_delta > (max_delta - current_delta):
            if status_netz:
                incident_steine += steine_im_netz
            steine_im_netz = 1
            status_netz = netztest(False,0,iterated_energy)
            masse_im_netz = df1_sim["Mass_(kg)"][i]
            current_delta = abs((max_delta - current_delta)-iterated_delta)

        elif iterated_delta <= (max_delta - current_delta):
            steine_im_netz += 1
            status_netz = netztest(status_netz,masse_im_netz,iterated_energy)
            masse_im_netz += df1_sim["Mass_(kg)"][i]
            current_delta = current_delta + iterated_delta



print("Anzahl simulierte Steine: ",sim_count)
print("Anzahl incident Steine: ",incident_steine)
print(incident_steine/sim_count*100)

Anzahl simulierte Steine:  2000000
Anzahl incident Steine:  9
0.00045


In [277]:
steine_pro_jahr = (sim_count / anzahl_sim_jahre)
incident_prob_pro_stein = incident_steine/sim_count*100
incident_prob_pro_jahr = incident_prob_pro_stein * steine_pro_jahr
print(incident_prob_pro_jahr, "%")
print(incident_prob_pro_jahr /100)

0.12941712730994095 %
0.0012941712730994096
